In [ ]:
%load_ext autoreload
%autoreload 2
# Enable imports form top-level of project (edit top_level_path accordingly)
import os
import sys
top_level_path = os.path.abspath(os.path.join('..'))
if top_level_path not in sys.path:
	sys.path.append(top_level_path)

In [4]:
from pyqubo import Binary, Array, Num
import numpy as np
from dimod import ExactSolver
import neal
from longestpath import gen_average_degree_directed, gen_planted_path, StandardGraph

In [20]:
# graph = StandardGraph(5, [
# 	(0,1),
#     (1,2),
#     (2,3),
#     (3,4)
# ])

graph = gen_average_degree_directed(20, 3)

# max length
N = graph.vertices
M = N

p = M+1

matrix = graph.to_matrix()

In [21]:
vars = Array.create('x', shape=(M+1, N+1), vartype='BINARY')

import time

start = time.time()
serial_exp = Num(-p) * sum([(sum(block) - Num(1))**2 for block in vars]) + Num(0)
end = time.time()

print(f"Done with serial_exp : Took {end - start} seconds")

#Seems to output same matrix as below
# serial_exp = Num(0)

# for m in range(M+1):
# 	temp_exp = Num(-1)
# 	for n in range(N+1):
# 		temp_exp += vars[m][n]
# 	serial_exp += Num(-p) * temp_exp ** 2

no_repeat_blocks_exp = Num(0)

start = time.time()

for i in range(N):
	print(f"{i} / {N-1}")
	for j in range(M):

		no_repeat_blocks_exp += sum([vars[j][i] * vars[k][i] for k in range(j+1, M+1)])

# 		for k in range(j+1, M+1):
# 			no_repeat_blocks_exp += vars[j][i] * vars[k][i] * Num(-p)
no_repeat_blocks_exp *= Num(-p)

end = time.time()

print(f"Done with no_repeatblocks_exp : Took {end - start} seconds")

start = time.time()
edges_exp = Num(0)
for m in range(M):
	for i in range(N):
		for j in range(N):
			if i == j:
				continue
			
			if matrix[i][j]:
				edges_exp += vars[m][i] * vars[m+1][j]
			else:
				edges_exp += Num(-p) * vars[m][i] * vars[m+1][j]

for m in range(M):
	for i in range(N):
		edges_exp += Num(-p) * vars[m][N] * vars[m+1][i]

end = time.time()

print(f"Done with edges_exp : Took {end - start} seconds")

Done with serial_exp : Took 0.0010008811950683594 seconds
0 / 19
1 / 19
2 / 19
3 / 19
4 / 19
5 / 19
6 / 19
7 / 19
8 / 19
9 / 19
10 / 19
11 / 19
12 / 19
13 / 19
14 / 19
15 / 19
16 / 19
17 / 19
18 / 19
19 / 19
Done with no_repeatblocks_exp : Took 0.08051037788391113 seconds
Done with edges_exp : Took 0.14351224899291992 seconds


In [22]:
def to_matrix(exp):
	model = exp.compile()
	qubo, energy_offset = model.to_qubo(index_label=True)

	return np.matrix([[int(qubo.get((i,j), 0)) for j in range((M+1)*(N+1))] for i in range((M+1)*(N+1))])

total_exp = serial_exp + no_repeat_blocks_exp + edges_exp

exp = -total_exp

model = exp.compile()
bqm = model.to_bqm()

# print(to_matrix(total_exp))

start = time.time()

sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, num_reads=100)
decoded_samples = model.decode_sampleset(sampleset)
best_sample = min(decoded_samples, key=lambda x: x.energy)
print(best_sample.energy)

end = time.time()

print(f"Solving took {end - start} seconds")

# sampleset = ExactSolver().sample(bqm)
# decoded_samples = model.decode_sampleset(sampleset)
# best_sample = max(decoded_samples, key=lambda s: s.energy)
# print(best_sample.energy)

-16.0
Solving took 1.1202082633972168 seconds
